In [59]:
# exercise to creating a python script to process data and create publication-ready plots
# this script re-creates figures 1c and 1d from the paper: "Single human oocyte transcriptome analysis reveals distinct maturation stage-dependent pathways impacted by age"

# doi: https://doi.org/10.1111/acel.13360
# written by: Bella Pfeiffer
# written for: CDSBF-550, Boston University Bioinformatics

In [ ]:
# importing libraries
import os
import pandas as pd
import numpy as np
from sklearn.manifold import TSNE  # for dimensionality reduction, similar to Rtsne in R
from sklearn.preprocessing import StandardScaler  # like scale() in R
from sklearn.decomposition import PCA  # similar to prcomp() in R
from sklearn.neighbors import NearestNeighbors
from scipy.stats import ttest_ind  # like t.test() in R
import matplotlib.pyplot as plt  # plotting library - quite different from ggplot2!
import seaborn as sns
import csv

In [ ]:
# helper functions for data processing 
def load_and_parse_metadata(metadata_file):
    """
    reads and cleans up the metadata from geo. in R, this would typically be done
    using readLines() or read.delim(), but python needs a bit more manual parsing
    
    important note: we're specifically looking for geo-formatted metadata here,
    which has those characteristic !Sample_* headers
    """
    metadata = {}
    
    # reading the file line by line - in R we might use readLines() instead
    with open(metadata_file, 'r') as f:
        for line in f:
            # look for the sample names and maturation stages
            # note: in geo files, these have special prefixes
            if line.startswith('!Sample_source_name_ch1'):
                # strip out those pesky quotes that geo adds
                metadata['metadata_id'] = [s.strip('"') for s in line.strip().split('\t')[1:]]
            elif 'maturation stage:' in line:
                # get just the stage name, throwing away the rest
                metadata['stage'] = [s.split()[-1].strip('"') for s in line.strip().split('\t')[1:]]
                break
    
    # convert to a dataframe - similar to as.data.frame() in R
    metadata_df = pd.DataFrame(metadata)
    
    # let's see what stages we have
    print("\nMetadata summary - Stage distribution:")
    print(metadata_df['stage'].value_counts())
    return metadata_df

def normalize_data_seurat_like(counts_df):
    """
    this implements seurat's normalization approach in python
    
    key differences from R/seurat:
    - we use pandas operations instead of sparse matrices
    - log1p is used instead of log() + 1
    - the implementation is more explicit here than seurat's internal code
    """
    print("normalizing data seurat-style...")
    
    # first get library size factors (sum per column)
    # in R: colSums(counts_df)
    size_factors = counts_df.sum(axis=0)
    
    # get the scaling factor - in R: median(colSums(counts_df))
    median_total = np.median(size_factors)
    
    # normalize by library size and scale - similar to sweep() in R
    normalized = counts_df.div(size_factors) * median_total
    
    # log transform - in R we'd do log(x + 1)
    log_norm = np.log1p(normalized)
    return log_norm

def calculate_DE_stats(counts_df, stages):
    """
    calculates differential expression between stages
    
    this is similar to using DESeq2 or edgeR in R, but we're doing a simpler
    implementation here with just t-tests. in a real analysis you'd probably
    want to use a proper DE package that handles dispersion estimation
    """
    print("\nrunning differential expression analysis...")
    
    # clean up any leftover quotes in stage names
    stages = np.array([str(s).strip('"') for s in stages])
    
    # create masks for our groups - in R we'd use logical indexing
    group1_mask = stages == 'GV'
    group2_mask = stages == 'MII'
    
    # doing some sanity checking
    print(f"we have {sum(group1_mask)} GV samples and {sum(group2_mask)} MII samples")
    
    # store results for each gene
    results = []
    for gene in counts_df.index:
        # get expression for each group
        group1_expr = counts_df.loc[gene, group1_mask]
        group2_expr = counts_df.loc[gene, group2_mask]
        
        # calculate means (adding pseudocount to avoid log(0))
        mean1 = np.mean(group1_expr) + 1
        mean2 = np.mean(group2_expr) + 1
        
        # get log2 fold change
        log2fc = np.log2(mean2/mean1)
        
        # run t-test (using welch's t-test by setting equal_var=False)
        # this is like t.test(..., var.equal=FALSE) in R
        try:
            t_stat, p_val = ttest_ind(group1_expr, group2_expr, equal_var=False)
        except:
            p_val = 1.0  # if the test fails, be conservative
        
        # store everything we calculated
        results.append({
            'gene': gene,
            'log2FC': log2fc,
            'pvalue': p_val,
            '-log10p': -np.log10(p_val) if p_val > 0 else 0,
            'mean_GV': np.mean(group1_expr),
            'mean_MII': np.mean(group2_expr)
        })
    
    # convert results to dataframe - similar to do.call(rbind, results) in R
    df_results = pd.DataFrame(results)
    
    # print some diagnostic info
    print("\nquick summary of what we found:")
    print(f"looked at {len(df_results)} genes total")
    print("\nfold change distribution:")
    print(df_results['log2FC'].describe())
    print("\np-value distribution:")
    print(df_results['pvalue'].describe())
    
    return df_results

In [ ]:
# helper functions specific to plotting
def plot_volcano_enhanced(de_stats, output_file='volcano_plot.png'):
    """
    creates a publication-ready volcano plot
    
    this is trying to replicate something like what you'd get from EnhancedVolcano in R,
    but using matplotlib instead of ggplot2. the styling is a bit more manual here!
    """
    # set up the figure - in R/ggplot2 this would be handled automatically
    # we're making it wide and short for better proportions
    plt.figure(figsize=(16, 8))
    
    # thresholds for significance
    # note: fold change threshold of 2 means log2FC of 1
    fc_threshold = 2  # this would be log2FoldChange in DESeq2
    p_threshold = 0.01  # typical threshold for differential expression
    
    # calculate -log10(p) just once for efficiency
    # in R we might do this in the plotting call itself
    neg_log_p = -np.log10(de_stats['pvalue'])
    
    # create masks for different categories of genes
    # this is similar to dplyr::mutate() + case_when() in R
    signif_up = (de_stats['log2FC'] > np.log2(fc_threshold)) & (de_stats['pvalue'] < p_threshold)
    signif_down = (de_stats['log2FC'] < -np.log2(fc_threshold)) & (de_stats['pvalue'] < p_threshold)
    fc_only = (abs(de_stats['log2FC']) > np.log2(fc_threshold)) & (de_stats['pvalue'] >= p_threshold)
    p_only = (abs(de_stats['log2FC']) <= np.log2(fc_threshold)) & (de_stats['pvalue'] < p_threshold)
    
    # plot each category with different colors
    # in ggplot2 this would be done with a single geom_point() call using aes()
    plt.scatter(de_stats.loc[~(signif_up | signif_down | fc_only | p_only), 'log2FC'],
               neg_log_p[~(signif_up | signif_down | fc_only | p_only)],
               c='gray', alpha=0.5, s=15, label='NS')
    
    plt.scatter(de_stats.loc[fc_only, 'log2FC'],
               neg_log_p[fc_only],
               c='green', alpha=0.7, s=15, label='Log₂ FC')
    
    plt.scatter(de_stats.loc[p_only, 'log2FC'],
               neg_log_p[p_only],
               c='blue', alpha=0.7, s=15, label='p-value')
    
    plt.scatter(de_stats.loc[signif_up | signif_down, 'log2FC'],
               neg_log_p[signif_up | signif_down],
               c='red', alpha=0.7, s=15, label='p-value and log₂ FC')
    
    # set the axis limits - in ggplot2 this would be scale_x/y_continuous()
    plt.xlim(-6, 6)
    plt.ylim(0, 35)
    
    # add labels for the most significant genes
    # similar to ggrepel in R, but more manual here
    significant = signif_up | signif_down
    if any(significant):
        # find the most interesting genes to label
        label_candidates = de_stats[significant].copy()
        # combine fold change and p-value to rank genes
        label_candidates['score'] = abs(label_candidates['log2FC']) * (-np.log10(label_candidates['pvalue']))
        top_genes = label_candidates.nlargest(15, 'score')
        
        # add text labels with white background for legibility
        for idx, row in top_genes.iterrows():
            plt.annotate(idx, 
                        xy=(row['log2FC'], -np.log10(row['pvalue'])),
                        xytext=(5, 5), textcoords='offset points',
                        fontsize=8, color='black',
                        bbox=dict(facecolor='white', edgecolor='none', alpha=0.7, pad=0.1))
    
    # add reference lines - like geom_vline() and geom_hline() in ggplot2
    plt.axvline(x=np.log2(fc_threshold), color='gray', linestyle='--', alpha=0.3)
    plt.axvline(x=-np.log2(fc_threshold), color='gray', linestyle='--', alpha=0.3)
    plt.axhline(y=-np.log10(p_threshold), color='gray', linestyle='--', alpha=0.3)
    
    plt.grid(True, linestyle='--', alpha=0.3)
    
    # label the axes - like labs() in ggplot2
    plt.xlabel('avg log₂FC', fontsize=12)
    plt.ylabel('-Log₁₀ P', fontsize=12)
    
    # add some helpful annotations
    plt.text(0, 33, '|FC = 2|', ha='center', va='top', color='gray')
    plt.text(-4, 33, 'GV markers', ha='center', va='top')
    plt.text(4, 33, 'IVM-MII markers', ha='center', va='top')
    
    # add legend - much more manual than in ggplot2!
    plt.legend(frameon=True, fontsize=10, title=None, 
              bbox_to_anchor=(0.02, 0.98), loc='upper left')
    
    # make sure the dimensions stick
    plt.gcf().set_size_inches(16, 8)
    
    # save the plot - like ggsave() in R
    plt.savefig(output_file, dpi=300, bbox_inches='tight')
    plt.close()

def run_tsne_on_pca(pca_result): # not plotting but included here for convenience
    """
    runs t-sne on our pca results for visualization
    
    this is similar to using Rtsne in R, but scikit-learn's implementation
    has some different parameters and defaults
    """
    print("\nrunning t-sne (this might take a minute)...")
    
    # create the t-sne object with carefully chosen parameters
    tsne = TSNE(
        n_components=2,  # we want 2D visualization
        # adjust perplexity based on sample size - this is crucial!
        perplexity=min(30, pca_result.shape[0] - 1),  
        init='pca',  # initialize with pca for better stability
        random_state=42,  # for reproducibility
        learning_rate='auto',  # let t-sne figure out the best learning rate
        n_iter=2000,  # run for longer than default
        early_exaggeration=12.0,  # helps with better cluster separation
        metric='euclidean'  # standard distance metric
    )
    
    # run t-sne - this is where the magic happens!
    tsne_result = tsne.fit_transform(pca_result)
    
    # print some info about the results
    print(f"t-sne transform done! output shape: {tsne_result.shape}")
    print("coordinates range:", 
          f"X: [{tsne_result[:,0].min():.2f}, {tsne_result[:,0].max():.2f}]",
          f"Y: [{tsne_result[:,1].min():.2f}, {tsne_result[:,1].max():.2f}]")
    
    return tsne_result
def plot_pca(pca_result, stages, output_file='pca_plot.png'):
    """
    creates a pca plot colored by developmental stage
    
    this is similar to what you might get from doing:
    ggplot(pca_df, aes(PC1, PC2, color=stage)) + geom_point()
    but we're using matplotlib's more manual approach
    """
    print("\n=== starting pca plotting routine ===")
    print(f"working with {pca_result.shape[1]} principal components")
    print(f"we have {len(stages)} samples to plot")
    
    # create a square figure - good for pca plots
    plt.figure(figsize=(8, 8))
    
    # clean up the stage labels - r usually handles this more gracefully
    stages = np.array([str(s).strip().strip('"') for s in stages])
    print(f"found these stages in the data: {np.unique(stages)}")
    
    # plot each developmental stage
    # we're using specific colors that work well together
    # in R/ggplot2 this would be handled by scale_color_manual()
    for stage, color in zip(['GV', 'MII'], ['#4444AA', '#FF8C55']):  # nice blue and orange
        mask = stages == stage
        count = np.sum(mask)
        print(f"\nplotting {count} samples for {stage} stage")
        
        if count > 0:
            # print the spread of points - helpful for debugging
            print(f"points spread from:")
            print(f"   x: {pca_result[mask, 0].min():.2f} to {pca_result[mask, 0].max():.2f}")
            print(f"   y: {pca_result[mask, 1].min():.2f} to {pca_result[mask, 1].max():.2f}")
            
            # create the scatter plot - each stage gets its own color
            plt.scatter(pca_result[mask, 0], 
                       pca_result[mask, 1],
                       c=color, 
                       label=stage, 
                       alpha=0.9,  # slightly transparent
                       s=80,  # nice big points
                       edgecolor='white',  # white edges look clean
                       linewidth=0.5)
    
    # label the axes - these are the first two principal components
    plt.xlabel('PC1', fontsize=12)
    plt.ylabel('PC2', fontsize=12)
    plt.title('PCA Plot', fontsize=14)
    
    # add a light grid - helps with readability
    plt.grid(True, linestyle='--', alpha=0.3)
    
    # add a legend if we plotted any points
    # in ggplot2 this would happen automatically
    if len(plt.gca().collections) > 0:
        plt.legend(frameon=True, fontsize=12)
        print("\nsuccessfully added the legend")
    else:
        print("\nwarning: no points were plotted!")
    
    # add sample counts in top-left corner
    # this would be like annotate() in ggplot2
    plt.text(0.02, 0.98,
             f"GV: {sum(stages == 'GV')} samples\n"
             f"MII: {sum(stages == 'MII')} samples",
             transform=plt.gca().transAxes,
             verticalalignment='top',
             fontsize=10,
             bbox=dict(facecolor='white', alpha=0.8, edgecolor='none'))
    
    # make sure everything fits nicely
    plt.tight_layout()
    
    # save the plot with error handling
    try:
        plt.savefig(output_file, dpi=300, bbox_inches='tight')
        print("\nplot saved successfully!")
    except Exception as e:
        print(f"\noops! error saving plot: {e}")
    finally:
        plt.close()

def plot_tsne(tsne_result, stages):
    """
    creates a t-sne plot colored by developmental stage
    
    this is very similar to our pca plot, but using t-sne coordinates
    in R this would look almost identical but use different input data
    """
    print("\n=== setting up t-sne visualization ===")
    
    # do some sanity checking on our input data
    print("checking our inputs:")
    print(f"t-sne coordinates shape: {tsne_result.shape}")
    print(f"number of samples: {len(stages)}")
    print(f"first few stages: {stages[:5]}")
    
    # clean up stage labels - always good to be thorough!
    stages = np.array([str(s).strip().strip('"') for s in stages])
    print("\nafter cleaning, found these stages:")
    unique_counts = pd.Series(stages).value_counts()
    print(unique_counts)
    
    # create a square plot - just like with pca
    plt.figure(figsize=(10, 10))
    
    # plot each stage with nice colors
    # using the same color scheme as pca for consistency
    colors = {'GV': '#4444AA', 'MII': '#FF8C55'}
    for stage in ['GV', 'MII']:
        mask = stages == stage
        count = np.sum(mask)
        print(f"\nworking on {stage} stage:")
        print(f"found {count} samples")
        
        if count > 0:
            # check the spread of points
            print(f"points spread from:")
            print(f"X: [{tsne_result[mask, 0].min():.2f} to {tsne_result[mask, 0].max():.2f}]")
            print(f"Y: [{tsne_result[mask, 1].min():.2f} to {tsne_result[mask, 1].max():.2f}]")
            
            # plot the points for this stage
            plt.scatter(tsne_result[mask, 0],
                       tsne_result[mask, 1],
                       c=colors[stage],
                       label=stage,
                       alpha=0.8,
                       s=100)  # slightly bigger points than pca
        else:
            print(f"no samples found for {stage}!")
    
    # label everything clearly
    plt.xlabel('t-SNE 1', fontsize=12)
    plt.ylabel('t-SNE 2', fontsize=12)
    plt.title('t-SNE Visualization of Gene Expression', fontsize=14)
    
    # add a legend if we have data
    if len(plt.gca().collections) > 0:
        plt.legend(frameon=True, fontsize=12)
        print("\nlegend added successfully")
    else:
        print("\nwarning: couldn't add legend - no points plotted!")
    
    # add sample counts just like in pca plot
    plt.text(0.02, 0.98,
             f"GV: {sum(stages == 'GV')} samples\n"
             f"MII: {sum(stages == 'MII')} samples",
             transform=plt.gca().transAxes,
             verticalalignment='top',
             fontsize=10)
    
    plt.tight_layout()
    
    # save the plot with error checking
    try:
        plt.savefig('tsne_plot_debug.png', dpi=300, bbox_inches='tight')
        print("\nplot saved successfully!")
    except Exception as e:
        print(f"\noops! error saving plot: {e}")
    finally:
        plt.close()
        

In [ ]:
# more data analysis 
def process_raw_dataset(counts_file, mapping_file, metadata_file):
    """
    loads and combines all our raw data files into a single analysis-ready dataset
    
    in R/Bioconductor, this would often be handled by functions like:
    readRDS() for counts
    read.delim() for mapping
    and specialized parsers for GEO metadata
    """
    print("\nstarting to load our raw data files...")
    
    # let's see where we are and what files we can see
    # helpful for debugging path issues!
    print(f"working in: {os.getcwd()}")
    print(f"i can see these files: {os.listdir()}")
    
    # load our three data files
    # in R we might use read.delim() for all of these
    counts_df = pd.read_csv(counts_file, sep='\t', index_col=0)  # gene counts
    mapping_df = pd.read_csv(mapping_file, sep='\t', header=None,  # sample mappings
                           names=['counts_id', 'metadata_id'])
    metadata_df = load_and_parse_metadata(metadata_file)  # sample metadata
    
    # clean up metadata - R usually handles quotes better!
    metadata_df['stage'] = metadata_df['stage'].str.strip('"')
    
    # combine metadata with mapping info
    # this is like merge() or join() in R
    sample_info = pd.merge(mapping_df, metadata_df, on='metadata_id', how='inner')
    
    # get just the counts we want
    # in R: counts_df[, sample_info$counts_id]
    processed_df = counts_df[sample_info['counts_id']].copy()
    
    # extract stages for later use
    stages = list(sample_info['stage'])
    
    # print some useful info about what we found
    print("\nlet's see what we have:")
    print(pd.Series(stages).value_counts())
    print("\nunique stages:", np.unique(stages))
    
    return processed_df, stages

def process_data_seurat_like(counts_df, n_pcs=20):
    """
    processes raw counts data using steps similar to seurat
    
    this replicates the basic seurat workflow:
    1. normalize
    2. find variable features
    3. scale data
    4. run pca
    
    in R/seurat this would be:
    NormalizeData() %>%
    FindVariableFeatures() %>%
    ScaleData() %>%
    RunPCA()
    """
    # step 1: normalize the data
    normalized_data = normalize_data_seurat_like(counts_df)
    
    # step 2: find variable features to focus on
    var_features = find_variable_features_seurat_like(normalized_data)
    
    # step 3: scale the variable features
    print("scaling our variable features...")
    data_to_scale = normalized_data.loc[var_features]
    # this is like scale() in R, but handles sparse data better
    scaled_data = StandardScaler().fit_transform(data_to_scale.T)
    
    # step 4: run pca
    print(f"\nrunning pca, keeping {n_pcs} components...")
    pca = PCA(n_components=n_pcs)
    pca_result = pca.fit_transform(scaled_data)
    
    # see how much variance we captured
    var_explained = pca.explained_variance_ratio_.sum()
    print(f"our {n_pcs} PCs explain {var_explained:.2%} of the variance")
    
    return pca_result, var_features, normalized_data

In [ ]:
def main():
    """
    main analysis workflow - this ties everything together!
    """
    print("let's analyze some gene expression data!")
    
    # first, load and prep our data
    # in real life, you'd want to make these paths configurable
    counts_df, stages = process_raw_dataset(
        counts_file="/Users/bellap/Desktop/cdsbf550/project2_v2/GSE158802_counts.tsv",
        mapping_file="/Users/bellap/Desktop/cdsbf550/project2_v2/GSE158802_samples.tsv",
        metadata_file="/Users/bellap/Desktop/cdsbf550/project2_v2/GSE158802_series_matrix.txt"
    )
    
    # let's see what we got
    print("\nquick data check:")
    print(f"our counts matrix is {counts_df.shape}")
    print(f"we have {len(stages)} samples")
    print("samples per stage:", pd.Series(stages).value_counts())
    
    # make sure everything is numeric
    # in R this would often be automatic
    counts_df = counts_df.astype(float)
    
    # filter out genes with zero counts
    # in R: counts_df[rowSums(counts_df) > 0, ]
    non_zero_genes = (counts_df.sum(axis=1) > 0)
    counts_df = counts_df.loc[non_zero_genes]
    print(f"after filtering, we have {counts_df.shape[0]} genes")
    
    # run our seurat-like processing pipeline
    print("\nstarting main analysis pipeline...")
    pca_result, var_features, normalized_data = process_data_seurat_like(counts_df)
    
    # check our pca results
    print("\npca output check:")
    print(f"shape: {pca_result.shape}")
    print("first few coordinates:")
    print(pca_result[:5, :2])
    
    # create all our visualizations
    print("\ngenerating plots...")
    plot_pca(pca_result, stages)
    
    # run t-sne on our pca results
    print("\nrunning t-sne for additional visualization...")
    tsne_result = run_tsne_on_pca(pca_result)
    plot_tsne(tsne_result, stages)

    # finally, run differential expression
    print("\nlooking for differentially expressed genes...")
    de_stats = calculate_DE_stats(normalized_data, stages)
    plot_volcano_enhanced(de_stats)
    
    print("\nall done! check the output directory for plots")

In [ ]:
# run code!
if __name__ == "__main__":
    main()